In [49]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.pinecone import Pinecone
import pinecone
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [50]:
#os.environ['ANTHROPIC_API_KEY'] = getpass("Enter Anthropic key:")
#os.environ['OPENAI_API_KEY'] = getpass("Enter OpenAI API key:")
#os.environ['PINECONE_API_KEY'] = getpass("Enter Pinecone API key:")
#os.environ['PINECONE_ENV'] = getpass("Enter Pinecone Environment:")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_API_ENV')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [51]:
# initialize embedding
model = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model,
    openai_api_key=OPENAI_API_KEY
)

In [52]:
text_field = "symptoms"
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

index_name = "medical-qa-search"
index = pinecone.Index(index_name)

# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [53]:
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [54]:
rag_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4',
    temperature=0.0
)

In [55]:
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [58]:
user_question = input("Please provide the symptoms here :")
result = qa_chain(user_question)

Please provide the symptoms here : What are the symptoms of alcoholic hepatitis?


In [59]:
result['result']

'The symptoms of alcoholic hepatitis include:\n\n1. Yellowing of the skin and whites of the eyes (jaundice)\n2. Malnutrition\n3. Loss of appetite\n4. Nausea and vomiting\n5. Abdominal tenderness\n6. Fever, often low grade\n7. Fatigue and weakness\n8. Fluid accumulation in your abdomen (ascites)\n9. Confusion and behavior changes due to a buildup of toxins normally broken down and eliminated by the liver\n10. Kidney and liver failure\n\nThese symptoms are mentioned in the context: "The most common sign of alcoholic hepatitis is yellowing of the skin and whites of the eyes (jaundice). Other signs and symptoms include: Malnutrition is common in people with alcoholic hepatitis. Drinking large amounts of alcohol suppresses the appetite, and heavy drinkers get most of their calories from alcohol. Additional signs and symptoms that occur with severe alcoholic hepatitis include: Loss of appetite, Nausea and vomiting, Abdominal tenderness, Fever, often low grade, Fatigue and weakness, Fluid acc

In [47]:
result['source_documents'][0].page_content

'[\'\', \'Shingles is characterized by pain or a tingling sensation in a limited area on one side of the face or torso, followed by a red rash with small, fluid-filled blisters.\', \'\', \'The signs and symptoms of shingles usually affect only a small section of one side of your body. These signs and symptoms may include:\', \'Some people also experience:\', \'Pain is usually the first symptom of shingles. For some, it can be intense. Depending on the location of the pain, it can sometimes be mistaken for a symptom of problems affecting the heart, lungs or kidneys. Some people experience shingles pain without ever developing the rash.\', \'Most commonly, the shingles rash develops as a stripe of blisters that wraps around either the left or right side of your torso. Sometimes the shingles rash occurs around one eye or on one side of the neck or face.\', \'Contact your doctor promptly if you suspect shingles, but especially in the following situations:\', \'\', \'\', \'\', \'\', \'\', \